In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from olist.seller import Seller

%load_ext autoreload
%autoreload 2

# SELLER P&L ANALYSIS

🎯 The goal of this exercise is to compute the P&L associated to each seller  

For each `seller_id` we need three items:  

- The **revenue**:
 - 10% fee on sales
 - 80 BRL per month on Olist


- The **cost**:
 - Review cost according to `{1: 100, 2: 50, 3: 40, 4: 0, 5: 0}` with review score as key and cost in BRL as value


- The **profit** made by Olist

💡 Let's not start from scratch  
❓ Import your seller training set and investigate what you already have.


In [5]:
# YOUR CODE HERE
sellers = Seller().get_training_data()
sellers.head()

,seller_id,seller_city,seller_state,delay_to_carrier,wait_time,date_first_sale,date_last_sale,months_on_olist,n_orders,quantity,quantity_per_order,sales,share_of_one_star,share_of_five_stars,review_score
0,3442f8959a84dea7ee197c632cb2df15,campinas,SP,0.000000,13.018588,2017-05-05 16:25:11,2017-08-30 12:50:19,4.0,3,3,1.000,218.70,1,1,3.000000
1,d1b65fc7debc3361ea86b5f14c68d2e2,mogi guacu,SP,0.000000,9.065716,2017-03-29 02:10:34,2018-06-06 20:15:21,14.0,40,41,1.025,11703.07,2,30,4.560976
2,ce3ad9de960102d0677a81f5d0bb7b2d,rio de janeiro,RJ,0.000000,4.042292,2018-07-30 12:44:49,2018-07-30 12:44:49,0.0,1,1,1.000,158.00,0,1,5.000000
3,c0f3eea2e14555b6faeea3dd58c1b1c3,sao paulo,SP,0.000000,5.667187,2018-08-03 00:44:08,2018-08-03 00:44:08,0.0,1,1,1.000,79.99,0,1,5.000000
4,51a04a8a6bdcb23deccc82b0b80742cf,braganca paulista,SP,3.353727,35.314861,2017-11-14 12:15:25,2017-11-14 12:15:25,0.0,1,1,1.000,167.99,1,0,1.000000


❓ What is missing?  
Write down a strategy to get your missing columns  
Re-use as much of what has already been coded in `seller.py` as possible

# Your pseudo-code

- Calculate the revenue

- Calculate the costs

- Substract the former from the former

❓ Make a copy of `seller.py`, rename it `seller_updated.py` and update it accordingly

> YOUR ANSWER HERE

❓ Compute seller profits

In [10]:
sellers["review_cost"] = np.floor(sellers["review_score"]).map({1: 100, 2: 50, 3: 40, 4: 0, 5: 0})
sellers.head()

,seller_id,seller_city,seller_state,delay_to_carrier,wait_time,date_first_sale,date_last_sale,months_on_olist,n_orders,quantity,quantity_per_order,sales,share_of_one_star,share_of_five_stars,review_score,review_cost
0,3442f8959a84dea7ee197c632cb2df15,campinas,SP,0.000000,13.018588,2017-05-05 16:25:11,2017-08-30 12:50:19,4.0,3,3,1.000,218.70,1,1,3.000000,40
1,d1b65fc7debc3361ea86b5f14c68d2e2,mogi guacu,SP,0.000000,9.065716,2017-03-29 02:10:34,2018-06-06 20:15:21,14.0,40,41,1.025,11703.07,2,30,4.560976,0
2,ce3ad9de960102d0677a81f5d0bb7b2d,rio de janeiro,RJ,0.000000,4.042292,2018-07-30 12:44:49,2018-07-30 12:44:49,0.0,1,1,1.000,158.00,0,1,5.000000,0
3,c0f3eea2e14555b6faeea3dd58c1b1c3,sao paulo,SP,0.000000,5.667187,2018-08-03 00:44:08,2018-08-03 00:44:08,0.0,1,1,1.000,79.99,0,1,5.000000,0
4,51a04a8a6bdcb23deccc82b0b80742cf,braganca paulista,SP,3.353727,35.314861,2017-11-14 12:15:25,2017-11-14 12:15:25,0.0,1,1,1.000,167.99,1,0,1.000000,100


In [12]:
sellers.groupby("seller_id", as_index=False).agg({"review_cost":sum})

,seller_id,review_cost
0,0015a82c2db000af6aaaf3ae2ecb0532,40
1,001cca7ae9ae17fb1caed9dfb1094831,40
2,002100f778ceb8431b7a1020ff7ab48f,40
3,003554e2dce176b5555353e4f3555ac8,0
4,004c9cd9d87a3c30c522c48c4fc07416,0
...,...,...
2962,ffc470761de7d0232558ba5e786e57b7,0
2963,ffdd9f82b9a447f6f8d4b91554cc7dd3,0
2964,ffeee66ac5d5a62fe688b9d26f83f534,0
2965,fffd5413c0700ac820c7069d66d98c89,40


In [15]:
olist_monthly_fee = 80
sales_charge = 0.1
sellers["subscription_revenue"] = sellers["months_on_olist"].apply(lambda x: x * olist_monthly_fee)

In [14]:
sellers["sales_revenue"] = sellers["sales"].apply(lambda x: x * sales_charge)

In [16]:
sellers["revenue"] = np.add(sellers["subscription_revenue"], sellers["sales_revenue"])

In [19]:
sellers["profit"] = np.subtract(sellers["revenue"], sellers["review_cost"])
sellers.profit

0        301.870
1       2290.307
2         15.800
3          7.999
4        -83.201
          ...   
2962      15.800
2963    1208.900
2964     455.004
2965     349.700
2966       1.250
Name: profit, Length: 2967, dtype: float64

> YOUR ANSWER HERE

❓ Load your updated DataFrame

In [20]:
# YOUR CODE HERE
sellers

,seller_id,seller_city,seller_state,delay_to_carrier,wait_time,date_first_sale,date_last_sale,months_on_olist,n_orders,quantity,quantity_per_order,sales,share_of_one_star,share_of_five_stars,review_score,review_cost,subscription_revenue,sales_revenue,revenue,profit
0,3442f8959a84dea7ee197c632cb2df15,campinas,SP,0.000000,13.018588,2017-05-05 16:25:11,2017-08-30 12:50:19,4.0,3,3,1.000000,218.70,1,1,3.000000,40,320.0,21.870,341.870,301.870
1,d1b65fc7debc3361ea86b5f14c68d2e2,mogi guacu,SP,0.000000,9.065716,2017-03-29 02:10:34,2018-06-06 20:15:21,14.0,40,41,1.025000,11703.07,2,30,4.560976,0,1120.0,1170.307,2290.307,2290.307
2,ce3ad9de960102d0677a81f5d0bb7b2d,rio de janeiro,RJ,0.000000,4.042292,2018-07-30 12:44:49,2018-07-30 12:44:49,0.0,1,1,1.000000,158.00,0,1,5.000000,0,0.0,15.800,15.800,15.800
3,c0f3eea2e14555b6faeea3dd58c1b1c3,sao paulo,SP,0.000000,5.667187,2018-08-03 00:44:08,2018-08-03 00:44:08,0.0,1,1,1.000000,79.99,0,1,5.000000,0,0.0,7.999,7.999,7.999
4,51a04a8a6bdcb23deccc82b0b80742cf,braganca paulista,SP,3.353727,35.314861,2017-11-14 12:15:25,2017-11-14 12:15:25,0.0,1,1,1.000000,167.99,1,0,1.000000,100,0.0,16.799,16.799,-83.201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2962,98dddbc4601dd4443ca174359b237166,sarandi,PR,0.000000,8.230822,2018-07-14 03:10:19,2018-07-23 15:34:03,0.0,2,2,1.000000,158.00,0,2,5.000000,0,0.0,15.800,15.800,15.800
2963,f8201cab383e484733266d1906e2fdfa,palhoca,SC,0.000000,13.045797,2017-06-17 15:30:11,2018-08-11 19:55:17,14.0,10,10,1.000000,889.00,1,5,4.100000,0,1120.0,88.900,1208.900,1208.900
2964,74871d19219c7d518d0090283e03c137,sao paulo,SP,0.000000,3.606625,2018-04-05 17:09:46,2018-08-21 19:25:15,5.0,6,7,1.166667,550.04,0,7,5.000000,0,400.0,55.004,455.004,455.004
2965,e603cf3fec55f8697c9059638d6c8eb5,pelotas,RS,0.000000,6.657634,2017-02-09 02:43:17,2017-06-16 17:30:16,4.0,8,11,1.375000,297.00,0,7,4.454545,0,320.0,29.700,349.700,349.700


❓ Sort sellers by profit, and analyse their profitability: conclude on a possible strategy for Olist!

In [28]:
# YOUR CODE HERE
profir_per_seller = sellers.sort_values(by="profit", ascending=False)[["seller_id","profit"]]
profir_per_seller

,seller_id,profit
2504,4869f7a5dfa277a7dca6462dcf3b52b2,24387.263
868,53243585a1d6dc2643021fd1853d8905,23237.605
2358,4a3ca9315b744ce9f8e9374361493884,21607.292
534,fa1c13f2614d7b5c4749cbc52fecda94,21004.203
1132,7c67e1448b00f6e969d365cea6b010ab,20192.389
...,...,...
2254,c97aa4ee7420f937da13b7f9e2228b99,-97.010
2135,6614814a00d344b846ae209f95ee7e3f,-98.700
153,c1dde11f12d05c478f5de2d7319ad3b2,-98.750
260,cc1f04647be106ba74e62b21f358af25,-98.810
